In [ ]:
%load_ext Cython

In [ ]:
import json
from datetime import datetime
import optuna as op
from optuna.samplers import TPESampler
import scipy.sparse as sp
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Utils.recsys2022DataReader import *

In [ ]:
URM = createBigBumpURM()

In [ ]:
URM_train_list = []
URM_validation_list = []

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM2, train_percentage=0.85)

for k in range(3):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.85)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)

recommender_P3alpha_list = []
recommender_SLIMEl

In [ ]:
def objective(trial):

    MAP_List_Fold = []

    topK = trial.suggest_int("topK", 400, 800)
    alpha = trial.suggest_float("alpha", 0.6, 0.9)

    for index in range(len(URM_train_list)):
        recommender_P3alpha = P3alphaRecommender(URM_train_list[index], verbose=False)

        recommender_P3alpha.fit(alpha=alpha, topK=topK)

        recommender_P3alpha_list.append(recommender_P3alpha)

        evaluator_validation = EvaluatorHoldout(URM_validation_list[index], cutoff_list=[10])

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender_P3alpha)

        MAP_List_Fold.append(result_dict.iloc[0]["MAP"])

    return sum(MAP_List_Fold) / len(MAP_List_Fold)